In [3]:
import pandas as pd
import glob
import pandas as pd
import json
import re
import pprint
# 读取所有 parquet 文件
data_name = "10G_data"



In [4]:

all_files = glob.glob(f"/media/sdb/zhurongjiang/Data_mining/{data_name}/*.parquet")
df = pd.concat([pd.read_parquet(file) for file in all_files], ignore_index=True)
data_name = data_name+"_results"

In [5]:

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = '/media/sdb/zhurongjiang/Data_mining/fronts/SourceHanSansSC-Regular.otf'  # 字体文件路径
my_font = fm.FontProperties(fname=font_path)
rows_with_missing = df.isnull().any(axis=1).sum()
print(f"存在缺失值的行数: {rows_with_missing}，占比: {rows_with_missing / len(df) * 100:.2f}%")


存在缺失值的行数: 0，占比: 0.00%


In [6]:
df.head(10)
#前一万数据



,id,timestamp,user_name,chinese_name,email,age,income,gender,country,chinese_address,purchase_history,is_active,registration_date,credit_score,phone_number
0,1,2025-01-09T01:38:20+00:00,UZPFPZJ,彭敏,xtnlkqsb@qq.com,36,73000.0,女,俄罗斯,广西壮族自治区绍兴和谐路152号2单元1384,"{""average_price"":15.940000000000001,""category""...",False,2024-10-02,423,918-668-7857
1,2,2023-07-08T22:53:52+00:00,UEHSG,高杰,rkpktrqz@outlook.com,58,223000.0,女,巴西,黑龙江省大连建设路127号1单元1835,"{""average_price"":563.4100000000001,""category"":...",False,2021-03-19,567,205-503-3300
2,3,2023-12-31T20:00:57+00:00,fxuujvnk,姜子轩,vwnquvla@outlook.com,88,858000.0,男,美国,浙江省厦门繁荣路15号5单元1442,"{""average_price"":669.34,""category"":""书籍"",""items...",False,2022-05-07,767,673-105-7503
3,4,2023-03-22T11:12:02+00:00,DDERCI,梁云,jpajekzz@qq.com,61,485000.0,男,德国,四川省宁波上海路30号6单元2134,"{""average_price"":637.66,""category"":""书籍"",""items...",False,2020-07-08,587,387-482-7104
4,5,2024-08-10T11:43:08+00:00,RTABMQKQLG,钱俊,haqlhpmb@163.com,33,437000.0,男,中国,四川省成都康乐路181号6单元391,"{""average_price"":505.0,""category"":""家居"",""items""...",False,2023-10-12,404,602-478-3001
5,6,2023-09-17T15:22:24+00:00,ypngbve,魏佳,xqujsqso@126.com,74,849000.0,男,中国,江西省台州工业路101号2单元1623,"{""average_price"":852.49,""category"":""食品"",""items...",False,2021-09-01,672,455-232-7578
6,7,2023-02-28T06:25:58+00:00,KBFYEUSBU,郝娜,cpqokuem@163.com,66,49000.0,女,法国,云南省惠州西湖路88号5单元139,"{""average_price"":328.78000000000003,""category""...",False,2020-05-16,624,998-655-1842
7,8,2023-12-08T19:45:29+00:00,kjmkgzk,贾玲,cjuedkaq@hotmail.com,90,522000.0,女,英国,上海市哈尔滨南京路100号5单元627,"{""average_price"":285.22,""category"":""食品"",""items...",False,2022-03-14,781,814-208-5773
8,9,2023-10-02T11:52:32+00:00,KYORF,赵俊,yxczxuqc@outlook.com,82,445000.0,女,英国,安徽省郑州朝阳路170号3单元1862,"{""average_price"":377.29,""category"":""家居"",""items...",False,2021-10-06,731,882-559-5557
9,10,2024-08-17T07:00:17+00:00,EOCGW,朱勇,ubrcnydq@126.com,20,189000.0,男,美国,澳门特别行政区苏州文化路32号5单元787,"{""average_price"":518.86,""category"":""家居"",""items...",False,2023-10-28,317,975-821-6895


In [7]:


# 提取函数重复定义（用于处理地址中提取省/市）
with open("pc.json", "r", encoding="utf-8") as f:
    province_city_dict_pc = json.load(f)
    
# for key,value in province_city_dict_pc.items():
#     print(key,value)
    
def extract_province_and_suffix(address):
    # 匹配省/直辖市/自治区/特别行政区
    prov_match = re.match(r'^(.{2,10}?(省|市|自治区|特别行政区))', address)
    
    if prov_match:
        province = prov_match.group(1)
        
        remaining = address[len(province):]
        

        # 取紧跟在省名之后的两个字
        suffix = remaining[:2] if len(remaining) >= 2 else remaining
        return pd.Series([province, suffix])
    
    return pd.Series([None, None])
# 应用提取并校验
# 提取省份和城市前缀
df[['province_extracted', 'city_extracted']] = df['chinese_address'].apply(extract_province_and_suffix)

# 校验省市是否匹配
def is_valid_pc(row):
    prov, city = row['province_extracted'], row['city_extracted']
    if pd.isna(prov) or pd.isna(city):
        return False
    valid_cities = province_city_dict_pc.get(prov, [])
    for valid_city in valid_cities:
        if city in valid_city or valid_city in city:
            return True
    return False

df['province_city_match'] = df.apply(is_valid_pc, axis=1)

# 保存不匹配的行
invalid_rows_pc = df[df['province_city_match'] == False][['chinese_address', 'province_extracted', 'city_extracted']]
output_path_pc = f"{data_name}/province_city_mismatch_with_user_pc_json.csv"
invalid_rows_pc.to_csv(output_path_pc, index=False)

# ✅ 从原始数据中剔除不匹配行
df = df[df['province_city_match'] == True].copy()

# output_path_pc

In [8]:
def detect_outliers(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    return ((series < lower) | (series > upper))

for col in ['age', 'income', 'credit_score']:
    outliers = detect_outliers(df[col])
    print(f"{col} 异常值数量：{outliers.sum()}，占比：{outliers.mean() * 100:.2f}%")


age 异常值数量：0，占比：0.00%
income 异常值数量：0，占比：0.00%
credit_score 异常值数量：0，占比：0.00%


In [9]:
before = len(df)
df = df[(df['age'] >= 0) & (df['income'] >= 0)]
after = len(df)
print(f"清洗数据：删除了 {before - after} 条记录，占总数 {(before - after) / before:.2%}")


清洗数据：删除了 0 条记录，占总数 0.00%


In [10]:

# 按年龄统计人数
age_counts = df['age'].value_counts().sort_index()

# 打印每个年龄对应的人数
print("各年龄用户数统计：")
print(age_counts)
age_counts.to_csv(f"{data_name}/age_count_stats.csv", header=["count"])

# 1. 用户年龄分布直方图
plt.figure(figsize=(10, 5))
sns.barplot(x=age_counts.index, y=age_counts.values, palette='viridis')
plt.title("distribution of user age")
plt.xlabel("age")
plt.ylabel("count")
plt.tight_layout()
plt.savefig(f"{data_name}/age_distribution.png")  # 保存图像
plt.close()  # 关闭图像，防止下一个图重叠


各年龄用户数统计：
age
18     34125
19     30500
20     31750
21     33125
22     33750
       ...  
96     33625
97     33500
98     31125
99     33250
100    33875
Name: count, Length: 83, dtype: int64


In [11]:
# 计算收入和信用评分的平均值
avg_income = df['income'].mean()
avg_credit_score = df['credit_score'].mean()
# 自定义收入区间（你可以根据实际数据调整）
income_bins = [0, 100000, 300000, 500000, 700000, 1000000]
income_labels = ['0-10w', '10w-30w', '30w-50w', '50w-70w', '70w-100w+']

# 分箱
df['income_range'] = pd.cut(df['income'], bins=income_bins, labels=income_labels, right=False)

# 统计每个收入区间人数
income_group_counts = df['income_range'].value_counts().sort_index()

print("各收入区间人数：")
print(income_group_counts)
# 自定义信用评分区间（可调整）
credit_bins = [0, 400, 600, 800, 1000]
credit_labels = ['低（0-400）', '中低（400-600）', '中高（600-800）', '高（800以上）']

df['credit_range'] = pd.cut(df['credit_score'], bins=credit_bins, labels=credit_labels, right=False)

# 统计每个信用评分区间人数
credit_group_counts = df['credit_range'].value_counts().sort_index()
income_group_counts.to_csv(f"{data_name}/income_group_counts.csv", header=["count"])
credit_group_counts.to_csv(f"{data_name}/credit_group_counts.csv", header=["count"])

print("各信用评分区间人数：")
print(credit_group_counts)

print(f"平均收入：{avg_income:.2f}")
print(f"平均信用评分：{avg_credit_score:.2f}")

plt.figure(figsize=(8, 5))
sns.scatterplot(x='income', y='credit_score', hue='gender', data=df)
plt.axvline(avg_income, color='red', linestyle='--', label=f'Avg Income: {avg_income:.0f}')
plt.axhline(avg_credit_score, color='blue', linestyle='--', label=f'Avg Credit Score: {avg_credit_score:.0f}')
plt.title("income vs credit score")
plt.xlabel("income")
plt.ylabel("credit_score")
plt.grid()
plt.legend(title='性别', loc='upper right')
plt.xlim(0, df['income'].max() * 1.1)
plt.ylim(0, df['credit_score'].max() * 1.1)
plt.tight_layout()
plt.savefig(f"{data_name}/income_vs_credit_score_with_avg.png")
plt.close()




各收入区间人数：
income_range
0-10w        279875
10w-30w      546250
30w-50w      546625
50w-70w      528125
70w-100w+    807250
Name: count, dtype: int64
各信用评分区间人数：
credit_range
低（0-400）       493625
中低（400-600）    982125
中高（600-800）    976625
高（800以上）       257750
Name: count, dtype: int64
平均收入：496240.72
平均信用评分：575.30


/tmp/ipykernel_348938/1798947481.py:44: UserWarning: Glyph 24615 (\N{CJK UNIFIED IDEOGRAPH-6027}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_348938/1798947481.py:44: UserWarning: Glyph 21035 (\N{CJK UNIFIED IDEOGRAPH-522B}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_348938/1798947481.py:44: UserWarning: Glyph 30007 (\N{CJK UNIFIED IDEOGRAPH-7537}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_348938/1798947481.py:44: UserWarning: Glyph 22899 (\N{CJK UNIFIED IDEOGRAPH-5973}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_348938/1798947481.py:44: UserWarning: Glyph 26410 (\N{CJK UNIFIED IDEOGRAPH-672A}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_348938/1798947481.py:44: UserWarning: Glyph 25351 (\N{CJK UNIFIED IDEOGRAPH-6307}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_348938/1798947481.py:44: UserWarning: Glyph 23450 (\N{CJK UNIFIED IDEOGRAPH-5B9A}) missing from current font.

In [12]:
# 3. 各国家用户分布柱状图
plt.figure(figsize=(10, 5))
#将国家中文名换为my_front字体
df['country'].value_counts().plot(kind='bar', color='skyblue')
plt.xticks(rotation=45, fontsize=10,fontproperties=my_font)
# 设置字体
plt.xticks(fontproperties=my_font)
plt.title("country user count",fontproperties=my_font)
plt.xlabel("country",fontproperties=my_font)
plt.ylabel("user count",fontproperties=my_font)
plt.tight_layout()
plt.savefig(f"{data_name}/user_count_by_country.png")
plt.close()

df['country'].value_counts()

country
中国      282000
日本      278375
印度      272875
巴西      270875
法国      270875
英国      270750
俄罗斯     269875
美国      267000
德国      263875
澳大利亚    263625
Name: count, dtype: int64

In [13]:
df['age_group'] = pd.cut(df['age'], bins=[0, 25, 45, 65, 100], labels=["青年", "中年", "中老年", "老年"])
df['income_level'] = pd.cut(df['income'], bins=3, labels=["低收入", "中收入", "高收入"])

# 从JSON格式购买信息中提取种类
df['purchase_category'] = df['purchase_history'].apply(lambda x: eval(x)['category'])


In [14]:
portrait = df.groupby(['age_group', 'income_level', 'gender'])['purchase_category'].value_counts().unstack().fillna(0)
# 计算每个组合的购买总数
portrait['total'] = portrait.sum(axis=1)
# 计算每个组合的购买比例
portrait = portrait.div(portrait['total'], axis=0)
#保存为csv
portrait.to_csv(f"{data_name}/user_portrait.csv")
# 打印用户画像
print("用户画像：")
print(portrait)
print(portrait.head())


用户画像：
purchase_category                    书籍        家居        服装      电子产品  \
age_group income_level gender                                           
青年        低收入          其他      0.142857  0.142857  0.142857  0.428571   
                       女       0.233146  0.196629  0.137640  0.205056   
                       未指定     0.000000  0.076923  0.153846  0.230769   
                       男       0.198880  0.162465  0.207283  0.221289   
          中收入          其他      0.318182  0.090909  0.000000  0.409091   
                       女       0.190323  0.183871  0.222581  0.174194   
                       未指定     0.166667  0.083333  0.333333  0.166667   
                       男       0.194444  0.182099  0.188272  0.212963   
          高收入          其他      0.083333  0.250000  0.166667  0.333333   
                       女       0.188679  0.194969  0.213836  0.163522   
                       未指定     0.157895  0.368421  0.210526  0.052632   
                       男       0.222826  0.18

In [15]:
plt.figure(figsize=(12, 6))
heatmap_data = df.pivot_table(index='age_group', columns='income_level', values='credit_score', aggfunc='mean')
sns.heatmap(heatmap_data, annot=True, cmap="YlGnBu")
plt.title("average credit score by age and income level")
plt.tight_layout()
plt.savefig(f"{data_name}/average_credit_score.png")
#保存在csv里面
plt.close()


/home/zhurongjiang/anaconda3/lib/python3.11/site-packages/seaborn/utils.py:80: UserWarning: Glyph 20302 (\N{CJK UNIFIED IDEOGRAPH-4F4E}) missing from current font.
  fig.canvas.draw()
/home/zhurongjiang/anaconda3/lib/python3.11/site-packages/seaborn/utils.py:80: UserWarning: Glyph 25910 (\N{CJK UNIFIED IDEOGRAPH-6536}) missing from current font.
  fig.canvas.draw()
/home/zhurongjiang/anaconda3/lib/python3.11/site-packages/seaborn/utils.py:80: UserWarning: Glyph 20837 (\N{CJK UNIFIED IDEOGRAPH-5165}) missing from current font.
  fig.canvas.draw()
/home/zhurongjiang/anaconda3/lib/python3.11/site-packages/seaborn/utils.py:80: UserWarning: Glyph 20013 (\N{CJK UNIFIED IDEOGRAPH-4E2D}) missing from current font.
  fig.canvas.draw()
/home/zhurongjiang/anaconda3/lib/python3.11/site-packages/seaborn/utils.py:80: UserWarning: Glyph 39640 (\N{CJK UNIFIED IDEOGRAPH-9AD8}) missing from current font.
  fig.canvas.draw()
/home/zhurongjiang/anaconda3/lib/python3.11/site-packages/seaborn/utils.py:80: U

In [16]:

heatmap_data.to_csv(f"{data_name}/average_credit_score.csv")